# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install citipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

626

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [9]:
# Base url
url = f"http://api.openweathermap.org/data/2.5/weather?units=imperial&appid={weather_api_key}&q=" #+ city

# List of city data
cityData = []

# Grouping variables
# Group
set_count = 1
# Record within group
record = 1

print("Beginning Data Retrieval")
print("--------------------------------")

for i, city in enumerate(cities):
    # group the cities in sets of 50
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record = 1
        
    # Output the record and group being processed
    print(f"Processing Record {record} of Set {set_count} | {city}")
    
    # Increase record number in the set
    record += 1
    
    # Create URL that includes city
    cityURL = url + city
    
    # Call the OpenWeather API
    try:
        cityWeather = requests.get(cityURL).json()
        
        # Parse out data
        lat = cityWeather["coord"]["lat"]
        lng = cityWeather["coord"]["lon"]
        maxTemp = cityWeather["main"]["temp_max"]
        humidity = cityWeather["main"]["humidity"]
        cloudiness = cityWeather["clouds"]["all"]
        windspeed = cityWeather["wind"]["speed"]
        date = cityWeather["dt"]
        
        cityData.append({"City": city,
                        "Lat": lat,
                        "Lng": lng,
                        "Max Temp": maxTemp,
                        "Humidity": humidity,
                        "Cloudiness": cloudiness,
                        "Wind Speed": windspeed,
                        "Date": date})
    except:
        print(f"{city} not found. Skipping...")
        pass

Beginning Data Retrieval
--------------------------------
Processing Record 1 of Set 1 | port hedland
Processing Record 2 of Set 1 | albany
Processing Record 3 of Set 1 | puerto asis
Processing Record 4 of Set 1 | ermelo
Processing Record 5 of Set 1 | atuona
Processing Record 6 of Set 1 | tubuala
Processing Record 7 of Set 1 | taolanaro
taolanaro not found. Skipping...
Processing Record 8 of Set 1 | rikitea
Processing Record 9 of Set 1 | mataura
Processing Record 10 of Set 1 | rio gallegos
Processing Record 11 of Set 1 | ostrovnoy
Processing Record 12 of Set 1 | weligama
Processing Record 13 of Set 1 | radyvyliv
Processing Record 14 of Set 1 | hermanus
Processing Record 15 of Set 1 | illoqqortoormiut
illoqqortoormiut not found. Skipping...
Processing Record 16 of Set 1 | isangel
Processing Record 17 of Set 1 | saint-philippe
Processing Record 18 of Set 1 | qaanaaq
Processing Record 19 of Set 1 | dakoro
Processing Record 20 of Set 1 | hobart
Processing Record 21 of Set 1 | bluff
Process

Processing Record 34 of Set 4 | yeppoon
Processing Record 35 of Set 4 | glasgow
Processing Record 36 of Set 4 | taunggyi
Processing Record 37 of Set 4 | ancud
Processing Record 38 of Set 4 | damavand
Processing Record 39 of Set 4 | riyadh
Processing Record 40 of Set 4 | khonsa
Processing Record 41 of Set 4 | alta floresta
Processing Record 42 of Set 4 | bilma
Processing Record 43 of Set 4 | carrollton
Processing Record 44 of Set 4 | pacific grove
Processing Record 45 of Set 4 | narsaq
Processing Record 46 of Set 4 | gotsu
Processing Record 47 of Set 4 | barrow
Processing Record 48 of Set 4 | pechenga
Processing Record 49 of Set 4 | kataysk
Processing Record 0 of Set 5 | svetlogorsk
Processing Record 1 of Set 5 | soe
Processing Record 2 of Set 5 | vaitupu
vaitupu not found. Skipping...
Processing Record 3 of Set 5 | huarmey
Processing Record 4 of Set 5 | naze
Processing Record 5 of Set 5 | poum
Processing Record 6 of Set 5 | te anau
Processing Record 7 of Set 5 | hun
Processing Record 8

Processing Record 20 of Set 8 | tutoia
Processing Record 21 of Set 8 | doctor pedro p. pena
doctor pedro p. pena not found. Skipping...
Processing Record 22 of Set 8 | paradwip
paradwip not found. Skipping...
Processing Record 23 of Set 8 | umm lajj
Processing Record 24 of Set 8 | east london
Processing Record 25 of Set 8 | sayalgudi
sayalgudi not found. Skipping...
Processing Record 26 of Set 8 | dujuma
dujuma not found. Skipping...
Processing Record 27 of Set 8 | bonavista
Processing Record 28 of Set 8 | port lincoln
Processing Record 29 of Set 8 | kurchum
Processing Record 30 of Set 8 | opuwo
Processing Record 31 of Set 8 | yulara
Processing Record 32 of Set 8 | si bun ruang
Processing Record 33 of Set 8 | nichlaul
Processing Record 34 of Set 8 | samandag
Processing Record 35 of Set 8 | arang
Processing Record 36 of Set 8 | talnakh
Processing Record 37 of Set 8 | houma
Processing Record 38 of Set 8 | krapivinskiy
Processing Record 39 of Set 8 | korla
Processing Record 40 of Set 8 | 

Processing Record 4 of Set 12 | srinivaspur
Processing Record 5 of Set 12 | nome
Processing Record 6 of Set 12 | muriwai beach
Processing Record 7 of Set 12 | garowe
Processing Record 8 of Set 12 | dingle
Processing Record 9 of Set 12 | yarada
Processing Record 10 of Set 12 | zemio
Processing Record 11 of Set 12 | collie
Processing Record 12 of Set 12 | okato
Processing Record 13 of Set 12 | lata
Processing Record 14 of Set 12 | tonj
Processing Record 15 of Set 12 | tulsipur
Processing Record 16 of Set 12 | tezu
Processing Record 17 of Set 12 | kampene
Processing Record 18 of Set 12 | ust-nera
Processing Record 19 of Set 12 | maragogi
Processing Record 20 of Set 12 | do gonbadan
Processing Record 21 of Set 12 | balabac
Processing Record 22 of Set 12 | honiara
Processing Record 23 of Set 12 | murchison
Processing Record 24 of Set 12 | wakkanai
Processing Record 25 of Set 12 | tomatlan
Processing Record 26 of Set 12 | kargopol
Processing Record 27 of Set 12 | nantucket
Processing Record 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [11]:
# Creates a data frame out of the raw data, and exports it as a csv
cityData_pd = pd.DataFrame(cityData)
cityData_pd.to_csv(r"../output_data/cities.csv")

In [12]:
# Displays the data frame
cityData_pd.head()
# Describe function. 7:22 time mark on video


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,port hedland,-20.3167,118.5667,64.40,36,0,10.36,1619124759
1,albany,42.6001,-73.9662,42.01,33,88,15.01,1619124940
2,puerto asis,0.5133,-76.5007,89.60,58,40,5.75,1619124759
3,ermelo,-26.5333,29.9833,52.52,62,89,3.36,1619124760
4,atuona,-9.8000,-139.0333,80.80,76,59,20.85,1619124689


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [7]:
#  Get the indices of cities that have humidity over 100%.


In [8]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression